In [2]:
import ee
import geemap

In [5]:
ee.Authenticate()
ee.Initialize()


Enter verification code:  4/1AVHEtk5NAEnnoayWU_2Qf5oaUTrApxayj3RVxLsU_1rB3R4y9LmPVDmGHz4



Successfully saved authorization token.


In [43]:
# Get Peru bounds
country_name = 'Peru'

country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', country_name)
)

r_lon = -80.458952
r_lat = -3.575189
r_poi = ee.Geometry.Point(r_lon, r_lat)
roi = ee.FeatureCollection(r_poi.buffer(10000))



start_date = '2022-01-01'
end_date = '2023-05-16'

In [91]:
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100


def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))




def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


s2cloudless_col = get_s2_sr_cld_col(country, start_date, end_date)

s2_sr_median = (s2cloudless_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median()).clipToCollection(country)


In [92]:
Map = geemap.Map()


# Display on a map

Map.addLayer(s2_sr_median, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1})

style = {'color': 'black', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, country_name)

roi_style = {'color': 'red', 'fillColor': '00000000'}
Map.addLayer(roi.style(**roi_style), {}, 'ROI')


Map.setCenter(r_lon, r_lat, 12);
Map

Map(center=[-3.575189, -80.458952], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

## Create quarterly images using the above procedure

In [93]:
# Dry season 1
q1_start_date = '2022-04-23'
q1_end_date = '2022-07-14'
q1_s2 = get_s2_sr_cld_col(country, q1_start_date, q1_end_date)
q1_s2_median = (q1_s2.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()).clipToCollection(country)

# Dry season 1
q2_start_date = '2022-07-15'
q2_end_date = '2022-10-15'
q2_s2 = get_s2_sr_cld_col(country, q2_start_date, q2_end_date)
q2_s2_median = (q2_s2.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()).clipToCollection(country)

# Dry season 2
q3_start_date = '2022-10-16'
q3_end_date = '2023-01-16'
q3_s2 = get_s2_sr_cld_col(country, q3_start_date, q3_end_date)
q3_s2_median = (q3_s2.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()).clipToCollection(country)

# Current flooding period in Tumbes https://weatherspark.com/y/18278/Average-Weather-in-Tumbes-Peru-Year-Round
flood_start_date = '2023-01-17'
flood_end_date = '2023-04-22'
flood_s2 = get_s2_sr_cld_col(country, flood_start_date, flood_end_date)
flood_s2_median = (flood_s2.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median()).clipToCollection(country)


## Calculate MNDWI for the composites

In [95]:
mndwi_q1 = q1_s2_median.normalizedDifference(['B3', 'B11']).rename('MNDWI')
mndwi_q2 = q2_s2_median.normalizedDifference(['B3', 'B11']).rename('MNDWI')
mndwi_q3 = q3_s2_median.normalizedDifference(['B3', 'B11']).rename('MNDWI')
mndwi_flood = flood_s2_median.normalizedDifference(['B3', 'B11']).rename('MNDWI')

In [96]:
Map = geemap.Map()

ndwi_vis = {'min': -1, 'max': 1, 'palette': 'RdBu'}
# Display on a map

Map.addLayer(mndwi_q1, ndwi_vis, 'Q1 MNDWI')
Map.addLayer(mndwi_q2, ndwi_vis, 'Q2 MNDWI')
Map.addLayer(mndwi_q3, ndwi_vis, 'Q3 MNDWI')
Map.addLayer(mndwi_flood, ndwi_vis, 'Flood MNDWI')

style = {'color': 'black', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, country_name)

Map.setCenter(r_lon, r_lat, 12);
Map

Map(center=[-3.575189, -80.458952], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

## Threshold the composites

In [106]:
mndwi_threshold = 0.35 # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6111878/
mndwi_water_q1 = mndwi_q1.gt(mndwi_threshold)
mndwi_water_q2 = mndwi_q2.gt(mndwi_threshold)
mndwi_water_q3 = mndwi_q3.gt(mndwi_threshold)
mndwi_water_flood = mndwi_flood.gt(mndwi_threshold)

In [ ]:
Map = geemap.Map()
Map.setCenter(-81.5, -3.6, 7)

Map.addLayer(mndwi_water_q1.selfMask(), {'palette': 'blue'}, 'mndwi_water_q1')
Map.addLayer(mndwi_water_q2.selfMask(), {'palette': 'red'}, 'mndwi_water_q2')
Map.addLayer(mndwi_water_q3.selfMask(), {'palette': 'yellow'}, 'mndwi_water_q3')
Map.addLayer(mndwi_water_flood.selfMask(), {'palette': 'orange'}, 'mndwi_water_flood')

Map.addLayer(country.style(**style), {}, country_name)
Map

## Calculate the flood extent from the max - min

In [ ]:
mndwi_flood_extent = mndwi_water_flood.subtract(mndwi_water_q3).gt(0).selfMask()

In [ ]:
Map = geemap.Map()
Map.setCenter(-81.5, -3.6, 10)

Map.addLayer(mndwi_flood_extent, {'palette': 'magenta'}, 'MNDWI Flood Extent')

Map.addLayer(country.style(**style), {}, country_name)
Map